In [1]:
import numpy, matplotlib, time, copy
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [2]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
        
    def get_n_best_books(self,n, bookValues):
        result = [];
        k=0
        for i in range(len(self.book_ids)):
            if bookValues[self.book_ids[i]] != 0:
                result.append(self.book_ids[i])
                k+=1
                if k == n:
                    return tuple(result)
        return tuple(result)
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
        
        

def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = [int(n) for n in content[1].split()]#tuple([int(n) for n in content[1].split()])
    libraries = []
    def book_sort(book):
        return bookValues[book]
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = tuple(sorted([int(id) for id in content[2+2*i+1].split()], key = book_sort, reverse=True))
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [3]:
def check_solution(D, libraries):
    print("D ",D)
    days = 0
    prev_books = set()
    for library in libraries:
#         print("#################")
#         print("Library id", library.id)
#         print("signup", library.signup_time)
#         print("Books per day", library.books_per_day)
#         print("all books", len(library.book_ids))
# #         print("##############")
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
#         elif len(library.book_ids) < (D-days)*library.books_per_day:
#             print("warning", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days <= D)

In [4]:

def get_score(file_path):
    (B,L,D),bookValues, libraries = process_file(file_path)
    unmodified_book_values = copy.copy(bookValues)
    chosen_libraries = []
    day = 0
    def book_sort(book):
        return bookValues[book]
    def library_score(lib):
        delta_time = D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, bookValues)
        sum_of_best_book_scores = sum([bookValues[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time
    
    while libraries:
        library_scores = [library_score(lib) for lib in libraries]
        next_library_index = library_scores.index(max(library_scores))
        next_library = libraries[next_library_index]
        day+=next_library.signup_time
        if day >= D:
            break
        next_library.book_ids = next_library.get_n_best_books((D-day)*next_library.books_per_day, bookValues)
        for book_id in next_library.book_ids:
            bookValues[book_id] = 0
        chosen_libraries.append(next_library)
        libraries.pop(next_library_index)
        
    print(len(chosen_libraries))
    score = 0
    for library in chosen_libraries:
        for idk in library.book_ids:
            score+=unmodified_book_values[idk]
    check_solution(D, chosen_libraries)
    return score
sum_score = 0
for file_path in file_paths:
    print(file_path)
    %time sum_score+=get_score(file_path)
print("SUM SCORE", sum_score)

a_example.txt
2
D  7
CPU times: user 465 µs, sys: 364 µs, total: 829 µs
Wall time: 463 µs
b_read_on.txt
90
D  1000
CPU times: user 1.09 s, sys: 10.4 ms, total: 1.1 s
Wall time: 1.12 s
c_incunabula.txt
1284
D  100000
CPU times: user 51.3 s, sys: 5.37 ms, total: 51.3 s
Wall time: 51.3 s
d_tough_choices.txt
15000
D  30001
CPU times: user 10min 34s, sys: 16.2 ms, total: 10min 34s
Wall time: 10min 34s
e_so_many_books.txt
131
D  200
CPU times: user 11.6 s, sys: 8 ms, total: 11.6 s
Wall time: 11.6 s
f_libraries_of_the_world.txt
17
D  700
CPU times: user 2.27 s, sys: 12 ms, total: 2.28 s
Wall time: 2.28 s
SUM SCORE 26344324


In [5]:
def get_score(file_path):
    (B,L,D),bookValues, libraries = process_file(file_path)
    unmodified_book_values = copy.copy(bookValues)
    day = 0
    def book_sort(book):
        return bookValues[book]
    def library_score(lib):
        delta_time = D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, bookValues)
        sum_of_best_book_scores = sum([bookValues[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time
#         return sum(sorted(lib.book_ids,key = book_sort, reverse=True)[:min(delta_time*lib.books_per_day,lib.size)])/lib.signup_time
    print(time.time())
    sorted_libraries = sorted(libraries,key=library_score, reverse=True)
    print(time.time())
    it=0;
    while it < len(sorted_libraries):
        next_library = libraries[it]
        day+=next_library.signup_time
        if day>=D:
            break
#         next_library.book_ids = set(sorted(next_library.book_ids, key = book_sort, reverse=True)[:min((D-day)*next_library.books_per_day,len(next_library.book_ids))])
        next_library.book_ids = next_library.get_n_best_books((D-day)*next_library.books_per_day, bookValues)
        for book_id in next_library.book_ids:
            bookValues[book_id] = 0
        sorted_libraries[it] = next_library
        it+=1
    print(time.time())
    score = 0
    for i in range(0,it):
        for idk in sorted_libraries[i].book_ids:
            score+=unmodified_book_values[idk]
    print("SCORE ",score)
    
    check_solution(D, sorted_libraries[:it])
    return score
sum_scores = 0
for file_path in file_paths:
    print(file_path)
    %time sum_scores +=get_score(file_path)
print("SUM SCORES", sum_scores)

a_example.txt
1608493536.5629113
1608493536.5629852
1608493536.5630777
SCORE  21
D  7
CPU times: user 1 ms, sys: 1 µs, total: 1 ms
Wall time: 640 µs
b_read_on.txt
1608493536.6104033
1608493536.6325564
1608493536.6411357
SCORE  4126100
D  1000
CPU times: user 83 ms, sys: 0 ns, total: 83 ms
Wall time: 81.7 ms
c_incunabula.txt
1608493536.7283137
1608493536.7725852
1608493536.7736084
SCORE  870640
D  100000
CPU times: user 134 ms, sys: 1 µs, total: 134 ms
Wall time: 133 ms
d_tough_choices.txt
1608493536.935606
1608493537.0172884
1608493537.0531936
SCORE  4109300
D  30001
CPU times: user 289 ms, sys: 8 ms, total: 297 ms
Wall time: 296 ms
e_so_many_books.txt
1608493537.252734
1608493537.3494456
1608493537.3504188
SCORE  755836
D  200
CPU times: user 276 ms, sys: 4 ms, total: 280 ms
Wall time: 279 ms
f_libraries_of_the_world.txt
1608493537.543406
1608493537.6718614
1608493537.6724494
SCORE  916843
D  700
CPU times: user 324 ms, sys: 0 ns, total: 324 ms
Wall time: 323 ms
SUM SCORES 10778740
